# LBM

<figure>
  <img src="https://raw.githubusercontent.com/autodesk/xlb/main/assets/wind_turbine.gif" width="500" height="340" alt="Wind turbine from XLB library">
  <figcaption><strong>Figure 1: Wind turbine from XLB library</strong></figcaption>
</figure>

The Lattice Boltzmann method is a relatively recent numerical technique in computational fluid dynamics (CFD). It offers features that make it extremely scalable on HPC systems. We will explore how some of the multi-GPU programming techniques presented in previous exercises can be applied to an LBM-based solver.

Next, we introduce basic LBM concepts for a 2D problem. For general information, see the Wikipedia page: [Lattice Boltzmann methods](https://en.wikipedia.org/wiki/Lattice_Boltzmann_methods).

Part of the code in these exercises has been adapted from the XLB library—a scalable, differentiable, open-source Python library for LBM developed by Autodesk. To run and profile more complex problems, check out its GitHub page: [XLB](https://github.com/Autodesk/XLB).


## Lattice Boltzmann Method

In the Lattice Boltzmann Method (LBM), the state of the fluid at each lattice node is represented by a set of particle distribution functions, $f_i(\mathbf{x}, t)$. Each $f_i$ can be thought of as the probability (or, more precisely, the expected number) of finding a fluid “particle” at position $\mathbf{x}$ and time $t$ that is moving along the discrete velocity direction $\mathbf{e}_i$. Rather than tracking individual molecules, LBM evolves these distributions through collision and streaming steps. In the collision step, the distributions at each node relax toward a local equilibrium—the Maxwell–Boltzmann distribution projected onto the discrete velocity set—ensuring that mass and momentum are conserved. In the streaming step, these post-collision distributions propagate to neighboring nodes, advecting the particle probabilities across the lattice.

Because each distribution $f_i$ carries a fraction of the local density and momentum, macroscopic properties are retrieved simply by summing over all directions:

$$
\rho(\mathbf{x}, t) \;=\; \sum_{i} f_i(\mathbf{x}, t),
\qquad
\rho(\mathbf{x}, t)\,\mathbf{u}(\mathbf{x}, t) \;=\; \sum_{i} f_i(\mathbf{x}, t)\,\mathbf{e}_i.
$$

This probabilistic interpretation makes LBM inherently statistical: collisions model how particle velocities redistribute toward equilibrium under local forces, and streaming moves those probabilities through space. It’s this combination of stochastic interpretation and discrete lattice mechanics that gives LBM both its physical fidelity and its remarkable parallel scalability.


## Starting Our First LBM Solver

Implementing and optimizing an LBM solver on multi-GPU systems using Warp promises to be an engaging challenge. However, to focus on core computational details without getting bogged down in tedious solver setup, we will leverage a local Python library called `lbm`. This library handles setting up most of the LBM data structures and problem configuration.

We'll first define the size of our 2D domain and the number of iterations we want to run.  
<!-- <img src="img/lattice-discretization.jpg" width="500" height="340"> -->

We are going to use some reference results to check the correctness of the solver when we apply a change to the code. The reference results are obtained with the following set of parameters:

`LBM Problem Parameters(nx=1024, ny=1024, num_steps=5000, Re=10000.0, prescribed_vel=0.5)`


In [1]:
import lbm
import time
import warp as wp
wp.clear_kernel_cache()
exercise_name = "00-lbm-singleGPU-aos"

params = lbm.Parameters(num_steps=5000,
                        nx=1024 ,
                        ny=1024 ,
                        prescribed_vel=0.5,
                        Re=10000.0)
print(params)


Warp 1.7.1 initialized:
   CUDA Toolkit 12.8, Driver 12.8
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA RTX A4000" (16 GiB, sm_86, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.7.1
LBM Problem Parameters(nx=1024, ny=1024, num_steps=5000, Re=10000.0, prescribed_vel=0.5)


The **Parameter** class stores some LBM constants, for example, the representation of the lattice. There are different lattices for LBM, and the following three are just examples.

<figure>
  <img src="img/lattices.jpg" alt="LBM lattice structures in 2D and 3D" width="500" height="340">
  <figcaption><strong>Figure 2: LBM lattice structures in 2D and 3D</strong></figcaption>
</figure>

In our case, we'll be using a D2Q9 lattice. The velocity vectors of the D2Q9 are represented by the **Parameter** via the **c_host** and **c_dev** fields. It is worth noting that the lattice includes a null vector representing the center of the cell.

In [2]:
params.c_host.shape

(2, 9)

In [3]:
print(f"D2Q9 \n{params.c_host}")

D2Q9 
[[ 0  0  0  1 -1  1 -1  1 -1]
 [ 0  1 -1  0  1 -1  0  1 -1]]


The **Parameter** class also includes functionality to retrieve the opposite direction in the lattice, as follows:

In [4]:
a_target_direction = params.c_host[:,2]
its_opposite = params.c_host[:,params.opp_indices_host[2]]
print(f"The opposite lattice direction of {a_target_direction} is {its_opposite}")

The opposite lattice direction of [ 0 -1] is [0 1]


## The LBM Domain

<figure>
  <img src="img/lattice-discretization.jpg" alt="Domain discretization in LBM" width="500" height="340">
  <figcaption><strong>Figure 3: Domain discretization in LBM</strong></figcaption>
</figure>

In LBM, we discretize the domain with a Cartesian background grid. To represent the probability distribution fields \(f_i\), we store, for each cell, one floating-point value per lattice direction. Therefore, we need to allocate a three-dimensional array where two dimensions represent the 2D spatial domain and the third represents the number of directions.

<figure>
  <img src="img/panda.png" alt="Exercise" width="7%" style="float: left; margin-right: 10px; margin-bottom: 10px;">
  <figcaption><strong>Exercise</strong>: Below is the shape of the 3D array for an Array-of-Structures layout.
  </figcaption>
</figure>


In [5]:
f_0 = wp.zeros((params.nx, params.ny, params.Q), dtype=wp.float64)
f_1 = wp.zeros((params.nx, params.ny, params.Q), dtype=wp.float64)

To abstract access to the population fields, we can define some read and write helper functions.


In [6]:
@wp.func
def read_field(field: wp.array3d(dtype=wp.float64), card: wp.int32, xi: wp.int32, yi: wp.int32):
    return field[xi, yi, card]

@wp.func
def write_field(field: wp.array3d(dtype=wp.float64), card: wp.int32, xi: wp.int32, yi: wp.int32,
                value: wp.float64):
    field[xi, yi, card] = value

## Some Helper Functions

In the following, we’ll use the **lbm** library to allocate additional fields for the macroscopic quantities. We aren’t concerned with these fields beyond visualization—indeed, the population fields are the only state variables required for LBM. We will also define several functions and kernels that serve as black boxes in our LBM solver.

In [7]:
# Initialize the memory
mem = lbm.Memory(params,
                 f_0=f_0,
                 f_1=f_1,
                 read=read_field,
                 write=write_field)

# Initialize the kernels
functions = lbm.Functions(params)
kernels = lbm.Kernels(params, mem)

Q = params.Q
D = params.D
bc_bulk = params.bc_bulk
c_dev = params.c_dev
dim_dev = params.dim_dev

## The LBM Operators

For the type of problem we will examine, an LBM iteration consists of three operators: **streaming**, the management of **boundary conditions**, and **collision**.  
We will not delve into the numerical methods details of these operators, as this is out of scope for this tutorial.  

<figure>
  <img src="img/stream-bc-collide.jpg" alt="LBM Loop" width="500" height="340">
  <figcaption><strong>Figure 4: LBM Loop</strong></figcaption>
</figure>

From a computational pattern perspective, **streaming** is a stencil operator: each cell reads information from its neighbors, as shown in the above picture. The remaining operators are **map** operators; they read and write only data associated with a single cell.  

For completeness, there are various techniques to implement LBM; we will be using a **two-population** method with a **pull** scheme.  

### Streaming

In the streaming operator, each cell iterates over its neighbors according to the lattice directions (`pull_ngh[d] = index[d] - c_dev[d, q]`) and reads the corresponding population, as illustrated above. The operator also needs to check that the computed neighbor location does not fall outside the domain.  


In [8]:
@wp.kernel
def stream(
        f_in: wp.array3d(dtype=wp.float64),
        f_out: wp.array3d(dtype=wp.float64),
):
    # Get the global index
    ix, iy = wp.tid()
    index = wp.vec2i(ix, iy)
    f_post = wp.vec(length=Q, dtype=wp.float64)

    for q in range(params.Q):
        pull_ngh = wp.vec2i(0, 0)
        outside_domain = False

        for d in range(D):
            pull_ngh[d] = index[d] - c_dev[d, q]

            if pull_ngh[d] < 0 or pull_ngh[d] >= dim_dev[d]:
                outside_domain = True
        if not outside_domain:
            f_post[q] = read_field(field=f_in, card=q, xi=pull_ngh[0], yi=pull_ngh[1])

    # Set the output
    for q in range(params.Q):
        write_field(field=f_out, card=q, xi=index[0], yi=index[1], value=f_post[q])

### Managing Boundary Conditions

We will use simple boundary conditions and rely on capabilities provided by the local `lbm` Python library. For each cell that is not flagged as `bulk`, we overwrite the population values using a `compute_boundary` Warp function, which we retrieve from an `lbm.Function` object. Warp functions can be passed around to compose kernels like any other Python object.

In [9]:
compute_boundaries = functions.get_apply_boundary_conditions()

In [10]:
@wp.kernel
def apply_boundary_conditions(
        bc_type_field: wp.array2d(dtype=wp.uint8),
        f_out: wp.array3d(dtype=wp.float64),
):
    # Get the global index
    ix, iy = wp.tid()

    bc_type = bc_type_field[ix, iy]
    if bc_type == bc_bulk:
        return

    f = compute_boundaries(bc_type)

    for q in range(params.Q):
        write_field(field=f_out, card=q, xi=ix, yi=iy, value=f[q])

### Collision

This is a *map* operator, and it operates in place on one LBM population field (`f`).  
The `f_post_stream` object is a vector of nine elements representing data read from memory; this vector should be considered a C array allocated on the stack.


In [11]:
compute_macroscopic = functions.get_macroscopic()
compute_equilibrium = functions.get_equilibrium()
compute_collision = functions.get_kbc()

In [12]:
@wp.kernel
def collide(
        f: wp.array3d(dtype=wp.float64),
        omega: wp.float64,
):
    # Get the global index
    ix, iy = wp.tid()
    # Get the equilibrium

    f_post_stream = wp.vec(length=Q, dtype=wp.float64)
    for q in range(params.Q):
        f_post_stream[q] = read_field(field=f, card=q, xi=ix, yi=iy)

    mcrpc = compute_macroscopic(f_post_stream)

    # Compute the equilibrium
    f_eq = compute_equilibrium(mcrpc)

    f_post_collision = compute_collision(f_post_stream, f_eq, mcrpc, omega)

    # Set the output
    for q in range(params.Q):
        write_field(field=f, card=q, xi=ix, yi=iy, value=f_post_collision[q])

## Problem Setup

We rely again on the `lbm` library to set up a typical CFD problem called **lid-driven cavity flow (LDC)**. The setup is simple: we consider a box filled with fluid, where on the lid we impose a tangential velocity. During the simulation, the lid’s velocity is propagated to the fluid.  
The following GIF shows the evolution of an LDC in a 3D domain:

<figure>
  <img src="img/ldc-3d.gif" alt="3D lid-driven cavity flow simulation" width="500" height="340">
  <figcaption><strong>Figure 5: A 3D LDC problem run in the XLB library</strong></figcaption>
</figure>


In [13]:
lbm.setup_LDC_problem(params=params, mem=mem)
lbm.export_setup(prefix=exercise_name, params=params, mem=mem)

Module lbm.kernels 218bc5d load on device 'cuda:0' took 129.18 ms  (compiled)
Module lbm.kernels fd66b4b load on device 'cuda:0' took 281.31 ms  (compiled)


<Figure size 640x480 with 0 Axes>

The following lines set up a 2D LDC problem and export the distribution of boundary types to a PNG file.  
Cells with a **wall** boundary condition are shown in green, the **lid** is shown in white, and **bulk** cells (fluid internal cells) are shown in black.

<figure>
  <img src="img/reference-00-lbm-intro-aos_bc_0000.png" alt="2D LDC boundary setup" width="250" height="340">
  <figcaption><strong>Figure 6: LDC setup</strong></figcaption>
</figure>


## LMB iteration

As shown in Figure 4, the LBM iteration is implemented by executing the `stream`, `apply_boundary_conditions`, and `collision` kernels. At the end of each iteration, the input and output 3D arrays are swapped for the next iteration.

In [14]:
def iterate():
    wp.launch(stream,
              dim=params.launch_dim,
              inputs=[mem.f_0, mem.f_1],
              device="cuda")

    wp.launch(apply_boundary_conditions,
              dim=params.launch_dim,
              inputs=[mem.bc_type, mem.f_1],
              device="cuda")

    wp.launch(collide,
              dim=params.launch_dim,
              inputs=[mem.f_1, params.omega],
              device="cuda")
    # Swap the fields
    mem.f_0, mem.f_1 = mem.f_1, mem.f_0

## LBM Main Loop

To focus completely on runtime performance without including compilation overhead, we first execute a warmup iteration and then start timing the main LBM loop. At the end, we print performance statistics and export the final velocity magnitude. Figure 6 shows the reference velocity for the LDC with the following parameters:


```
params = lbm.Parameters(num_steps=5000,
                        nx=1024 ,
                        ny=1024 ,
                        prescribed_vel=0.5,
                        Re=10000.0)
```

<figure>
  <img src="img/reference_00-lbm-intro-aos_u_5000.png" alt="LDC setup" width="250" height="340">
  <figcaption><strong>Figure 6: LDC velocity magnitude</strong></figcaption>
</figure>

In [15]:
# Warm up iteration
iterate()

# Wait for the warm-up to finish.
wp.synchronize()
# Start timer
start = time.time()
for it in range(params.num_steps):
    iterate()

wp.synchronize()
stop = time.time()

# Saving the velocity magnitude.
lbm.export_final(prefix=exercise_name, params=params, mem=mem, f=mem.f_0)




Module __main__ d0e9453 load on device 'cuda:0' took 936.04 ms  (compiled)
Module lbm.kernels 70eb1dc load on device 'cuda:0' took 416.07 ms  (compiled)


<Figure size 640x480 with 0 Axes>

## Performance

For LBM performance, we are interested in the time spent in the main loop, but even more importantly, its throughput. Usually, we look at the number of floating-point operations per second; however, we will focus on a simpler metric called MLUPS: Million Lattice Updates Per Second. This is the reference metric in the LBM community. In the next couple of LBM exercises, we'll consider MLUPS as the reference metric to compare different implementations.

In [16]:
# Printing some statistics.
elapsed_time = stop - start
mlups = params.compute_mlups(elapsed_time)
print(f"--------------------------------------------")
print(f"Exercise: {exercise_name}")
print(f"Main loop time: {elapsed_time:5.3f} seconds")
print(f"MLUPS:          {mlups:5.1f}")
print(f"--------------------------------------------")

--------------------------------------------
Exercise: 00-lbm-singleGPU-aos
Main loop time: 17.891 seconds
MLUPS:          293.1
--------------------------------------------


## Profiling

More information on the behavior of our application running on a GPU can be extracted by simply using a profiler. NVIDIA provides two main tools that are useful for our case: [Nsight Systems](https://developer.nvidia.com/nsight-systems) and [Nsight Compute](https://developer.nvidia.com/nsight-compute). The first provides system-wide information about the execution, while the second focuses on single-kernel, in-depth runtime analysis. We are going to use only Nsight Systems.  

We can run Nsight Systems directly from this notebook by enabling it as shown in the following figures:

<div style="display: flex; align-items: flex-start;">

  <figure style="margin-right: 5%; width: 30%; text-align: center;">
    <img src="img/activate-nsys-01.png" style="width: 100%;" />
    <figcaption><strong>Figure 8: Play with Nsight Systems (step 1)</strong></figcaption>
  </figure>

  <figure style="width: 30%; text-align: center;">
    <img src="img/activate-nsys-02.png" style="width: 100%;" />
    <figcaption><strong>Figure 9: Play with Nsight Systems (step 2)</strong></figcaption>
  </figure>

</div>

**Note:** The solver behavior will not change drastically from one iteration to the next. As the profiler will add some overhead to execution and capture a lot of information, we may want to reduce the number of solver iterations before running the profiler.  

Playing a notebook cell with the green play icon will automatically run the cell within the profiler and print performance statistics. The profiling session will also create a local file that you can download and inspect locally with the Nsight Systems UI:

<div style="display: flex; align-items: flex-start;">

  <figure style="margin-right: 5%; width: 20%; text-align: center;">
    <img src="img/activate-nsys-03.png" style="width: 100%;" />
    <figcaption><strong>Figure 10: Nsight Systems report folder</strong></figcaption>
  </figure>

  <figure style="width: 20%; text-align: center;">
    <img src="img/activate-nsys-04.png" style="width: 100%;" />
    <figcaption><strong>Figure 11: Download</strong></figcaption>
  </figure>

</div>

Once downloaded and loaded into the UI, you can zoom into the area of interest and inspect the application timeline:

<figure>
  <img src="img/aos-nsys.png" alt="Profiler timeline" width="100%">
  <figcaption><strong>Figure 12: Profiler timeline</strong></figcaption>
</figure>


<figure>
  <img src="img/panda.png" alt="Exercise" width="7%" style="float: left; margin-right: 10px; margin-bottom: 10px;">
  <figcaption><strong>Exercise</strong>: Try running the profiler and checking out the timeline.  
  </figcaption>
</figure>
